In [1]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import h5py
from src.benchmarkers import *
from src.benchmarkersV2 import *
import time
from tqdm import tqdm
from matplotlib import pyplot as plt
np.random.seed(0)

In [21]:
def print_name(name):
    print(name)

with h5py.File(r"C:\Users\Cristiano Lavoro\Desktop\benchmarks\outputs\v2\32\ds_10_core.h5", "r") as file:
    print(find_max_iterations(stringa,file))

#table = pq.read_table(r"C:\Users\Cristiano Lavoro\Desktop\benchmarks\outputs\v2\125\ds_100.parquet")
#image2 = np.array(table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("image_1_feature").values.field("image").to_pylist())
#image2np = np.frombuffer(image2[0], dtype=np.float64).reshape(3, 32, 32)
#image1 == image2np

example_0/image_feature/image1
example_0/image_feature/image2
example_0/image_feature/image3
example_0/image_feature/image4
3


In [2]:
#N = [10,100,200,300,500]
N = [10]
dimensions = [32,64,125,192]
selected_label = 10
iterations = 100

In [23]:
generator = Generator()

### Generation

In [24]:
for item in tqdm(N):
    for dim in dimensions:
        generator.create_dataset(item,f"outputs/v2/{dim}/ds_{item}",dim)
        generator.create_arrow_file(f"outputs/v2/{dim}/ds_{item}")
        generator.create_arrow_stream(f"outputs/v2/{dim}/ds_{item}")

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [12:35<00:00, 151.05s/it]


In [5]:
ClockBoundingBoxConversion().benchmark_parquet(r"C:\Users\rober\onedrive\desktop\projects\benchmarks\outputs\v2\32\ds",N,iterations)
ClockBoundingBoxConversion().benchmark_hdf5(r"C:\Users\rober\onedrive\desktop\projects\benchmarks\outputs\v2\32\ds",N,iterations,hdf5_driver = "core")

100%|██████████| 100/100 [00:00<00:00, 255.27it/s]


[array([[0.25  , 0.25  , 0.75  , 0.75  ],
       [0.25  , 0.25  , 0.75  , 0.75  ],
       [0.25  , 0.25  , 0.75  , 0.75  ],
       [0.0977, 0.2604, 0.8789, 0.7812],
       [0.0977, 0.2604, 0.8789, 0.7812],
       [0.25  , 0.25  , 0.75  , 0.75  ],
       [0.25  , 0.25  , 0.75  , 0.75  ],
       [0.25  , 0.25  , 0.75  , 0.75  ],
       [0.0977, 0.2604, 0.8789, 0.7812],
       [0.0977, 0.2604, 0.8789, 0.7812],
       [0.25  , 0.25  , 0.75  , 0.75  ],
       [0.25  , 0.25  , 0.75  , 0.75  ],
       [0.25  , 0.25  , 0.75  , 0.75  ],
       [0.0977, 0.2604, 0.8789, 0.7812],
       [0.0977, 0.2604, 0.8789, 0.7812],
       [0.25  , 0.25  , 0.75  , 0.75  ],
       [0.25  , 0.25  , 0.75  , 0.75  ],
       [0.25  , 0.25  , 0.75  , 0.75  ],
       [0.0977, 0.2604, 0.8789, 0.7812],
       [0.0977, 0.2604, 0.8789, 0.7812],
       [0.25  , 0.25  , 0.75  , 0.75  ],
       [0.25  , 0.25  , 0.75  , 0.75  ],
       [0.25  , 0.25  , 0.75  , 0.75  ],
       [0.0977, 0.2604, 0.8789, 0.7812],
       [0.0977,

100%|██████████| 100/100 [00:00<00:00, 137.64it/s]

[[np.float64(0.25), np.float64(0.25), np.float64(0.75), np.float64(0.75)], [np.float64(0.25), np.float64(0.25), np.float64(0.75), np.float64(0.75)], [np.float64(0.25), np.float64(0.25), np.float64(0.75), np.float64(0.75)], [np.float64(0.25), np.float64(0.25), np.float64(0.75), np.float64(0.75)], [np.float64(0.25), np.float64(0.25), np.float64(0.75), np.float64(0.75)], [np.float64(0.25), np.float64(0.25), np.float64(0.75), np.float64(0.75)], [np.float64(0.25), np.float64(0.25), np.float64(0.75), np.float64(0.75)], [np.float64(0.25), np.float64(0.25), np.float64(0.75), np.float64(0.75)], [np.float64(0.25), np.float64(0.25), np.float64(0.75), np.float64(0.75)], [np.float64(0.25), np.float64(0.25), np.float64(0.75), np.float64(0.75)], [np.float64(0.25), np.float64(0.25), np.float64(0.75), np.float64(0.75)], [np.float64(0.25), np.float64(0.25), np.float64(0.75), np.float64(0.75)], [np.float64(0.25), np.float64(0.25), np.float64(0.75), np.float64(0.75)], [np.float64(0.25), np.float64(0.25), 